# Time-series analysis with XArray and Zarr

Here, we'll introduce a couple more tools that add to our arsenal and perform a more realistic analysis of neuroscience data. 

We'll use two tools that integrate with the other things we've seen so far. 

The first is Zarr. This is primarily a file format that allows us to save data (e.g., time-series) in a way that supports optimal ingesting into a distributed cluster. 

The second is XArray. This is a Python library that supports the management of complex datasets, such as multi-channel time-series data from neuroscience experiments.

Let's start by importing XArray and using it to read some data from a GCS bucket

In [1]:
import gcsfs
import xarray as xr

This time we're pointing to another bucket that is publicly available and contains some data that Chris Holdgraf collected (as described [here](https://www.nature.com/articles/ncomms13654)).

In [2]:
fs = gcsfs.GCSFileSystem('holdgraf-ecog')

XArray knows how to read data stored as a zarr into an `XArray` `DataSet` object. To identify the GCS location, it is given a `GCSMap` object with the file-system as a pointer and with `check` and `create` both set to `False` (this is the read only mode).

In [3]:
gcsmap = gcsfs.mapping.GCSMap('holdgraf-ecog/sub-01-zarr', 
                              gcs=fs, 
                              check=False, 
                              create=False)

Once this map is provided to XArray it creates the Dataset

In [4]:
data = xr.open_zarr(gcsmap)

In [5]:
channels = list(data.keys())

In [6]:
# Setup a dask cluster
from dask.distributed import Client
from dask_kubernetes import KubeCluster

cluster = KubeCluster(n_workers=10)
cluster

In [7]:
client = Client(cluster)
client

Client Scheduler: tcp://10.36.13.22:35651 Dashboard: /user/arokem/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [8]:
from nitime import algorithms as tsa
from nitime import utils as tsu

The data was filtered between 0.1 and 1,000 Hz, so that's the sampling frequency:

In [9]:
Fs = 1000

In [10]:
N = data[channels[0]].shape[0]

In [12]:
NW = 4
bandwidth = NW * (2 * Fs) / N
tapers, eigs = tsa.dpss_windows(N, NW, 2 * NW - 1)

/srv/conda/envs/notebook/lib/python3.6/site-packages/nitime/utils.py:980: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  complex_result = (np.issubdtype(in1.dtype, np.complex) or
/srv/conda/envs/notebook/lib/python3.6/site-packages/nitime/utils.py:981: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  np.issubdtype(in2.dtype, np.complex))


In [13]:
tapers.shape, eigs.shape

((7, 1204042), (7,))

In [14]:
import numpy as np

In [18]:
def get_spectra(data, ch, tapers, eigs):
    dd = data[ch].chunk(data[ch].shape[0])
    tdata = tapers[None, :] * dd.values
    ss = tsa.fftpack.fft(tdata.squeeze())
    # We can set these to be adaptive:
    # weights, df = tsu.adaptive_weights(ss, eigs, sides='onesided')
    # Or take the square root of the eigenvalues
    weights = np.sqrt(eigs)
    df = None
    return ss, weights, df

In [19]:
def mt_coherence(data, ch1, ch2, tapers, eigs):
    sx, wx, dfx = get_spectra(data, ch1, tapers, eigs)
    sy, wy, dfy = get_spectra(data, ch2, tapers, eigs)
    sxy = tsa.mtm_cross_spectrum(sx, sy, (wx, wy),
                                 sides='onesided')
    sxx = tsa.mtm_cross_spectrum(sx, sx, (wx, wx),
                                 sides='onesided')
    syy = tsa.mtm_cross_spectrum(sy, sy, (wy, wy),
                                 sides='onesided')
    coh = np.abs(sxy) ** 2 / (sxx *  syy)
    
    # XXX Calculate jack-knife estimates of 95% confidence intervals
    
    return coh

In [20]:
coh_test = mt_coherence(data, channels[0], channels[1], tapers, eigs)

IndexError: too many indices for array

In [ ]:
debug

> /srv/conda/envs/notebook/lib/python3.6/site-packages/nitime/algorithms/spectral.py(626)mtm_cross_spectrum()
    624         # if weights.shape[-1] > 1 then make sure weights are truncated too
    625         if weights_x.shape[-1] > 1:
--> 626             weights_x = weights_x[tsl]
    627             weights_y = weights_y[tsl]
    628             denom = denom[tsl[1:]]



ipdb>  p tsl


(slice(None, None, None), slice(0, 602022, None))


ipdb>  weights_x.shape


(7,)
